# Bosphorus Vision - YOLO11 Fine-Tuning

**Dataset:** Custom Bosphorus dataset  
**Environment:** Google Colab with T4 GPU

| Parameter | Value |
|-----------|-------|
| Model | YOLO11s |
| Image Size | 1088 |
| Batch Size | Auto (-1) |
| Epochs | 50 |

## Step 1: Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
dataset_path = "/content/drive/MyDrive/Uskudar Uni Master/YOLO Model klasor/Custom Dataset/bogaz_v_1.v3i.yolov12"

if os.path.exists(dataset_path):
    print("Dataset found:", os.listdir(dataset_path))
else:
    print("Dataset NOT found:", dataset_path)

In [ ]:
# Install Ultralytics
!pip install -U ultralytics

from ultralytics import YOLO
import ultralytics
print(f"Ultralytics version: {ultralytics.__version__}")

In [ ]:
# Check GPU
import torch

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("No GPU detected - go to Runtime > Change runtime type > GPU")

## Step 2: Fix YAML Paths

In [ ]:
# Fix data.yaml paths to point to correct locations
import yaml
import os

dataset_path = "/content/drive/MyDrive/Uskudar Uni Master/YOLO Model klasor/Custom Dataset/bogaz_v_1.v3i.yolov12"
yaml_path = os.path.join(dataset_path, "data.yaml")

# Check what folders exist
print("Folders in dataset:")
for item in os.listdir(dataset_path):
    full_path = os.path.join(dataset_path, item)
    if os.path.isdir(full_path):
        # Check for images subfolder
        images_path = os.path.join(full_path, "images")
        if os.path.exists(images_path):
            count = len([f for f in os.listdir(images_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
            print(f"  {item}/images: {count} images")
        else:
            # Images directly in folder
            count = len([f for f in os.listdir(full_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
            print(f"  {item}: {count} images")

# Build correct paths based on actual folder structure
train_path = os.path.join(dataset_path, "train")
val_path = os.path.join(dataset_path, "valid")
test_path = os.path.join(dataset_path, "test")

# Check if images are in subfolders or directly in train/valid/test
if os.path.exists(os.path.join(train_path, "images")):
    train_path = os.path.join(train_path, "images")
if os.path.exists(os.path.join(val_path, "images")):
    val_path = os.path.join(val_path, "images")
if os.path.exists(os.path.join(test_path, "images")):
    test_path = os.path.join(test_path, "images")

# Read and update YAML
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

data['train'] = train_path
data['val'] = val_path
if os.path.exists(test_path) or os.path.exists(test_path.replace('/images', '')):
    data['test'] = test_path

with open(yaml_path, 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

# Verify
print("\nUpdated paths:")
for key in ['train', 'val', 'test']:
    if key in data and data[key]:
        exists = os.path.exists(data[key])
        if exists:
            count = len([f for f in os.listdir(data[key]) if f.endswith(('.jpg', '.jpeg', '.png'))])
            print(f"  {key}: OK ({count} images) - {data[key]}")
        else:
            print(f"  {key}: MISSING - {data[key]}")

In [ ]:
# Show class names
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

print(f"Classes ({data.get('nc', len(data.get('names', [])))}):")
names = data.get('names', [])
if isinstance(names, dict):
    for idx, name in names.items():
        print(f"  {idx}: {name}")
else:
    for idx, name in enumerate(names):
        print(f"  {idx}: {name}")

## Step 3: Training

In [ ]:
# Load model
from ultralytics import YOLO

MODEL_SIZE = "s"  # Options: n, s, m, l, x
model = YOLO(f"yolo11{MODEL_SIZE}.pt")
model.info()

In [ ]:
# Train
import os

dataset_path = "/content/drive/MyDrive/Uskudar Uni Master/YOLO Model klasor/Custom Dataset/bogaz_v_1.v3i.yolov12"
yaml_path = os.path.join(dataset_path, "data.yaml")

results = model.train(
    data=yaml_path,
    imgsz=1088,
    rect=True,
    batch=-1,
    epochs=50,
    patience=10,
    cache=True,
    workers=2,
    project="/content/drive/MyDrive/Uskudar Uni Master/YOLO Model klasor/runs",
    name="bosphorus_yolo11s_1088",
    amp=True,
    verbose=True,
    
    # Optimizer settings
    optimizer='AdamW',
    cos_lr=True,
    warmup_epochs=3,
    
    # Learning rate
    lr0=0.001,
    lrf=0.01,
)

print("Training complete!")

## Step 4: Inference

In [ ]:
# Load trained model
from ultralytics import YOLO
import os

best_model_path = "/content/drive/MyDrive/Uskudar Uni Master/YOLO Model klasor/runs/bosphorus_yolo11s_1088/weights/best.pt"

if os.path.exists(best_model_path):
    trained_model = YOLO(best_model_path)
    print(f"Loaded: {best_model_path}")
else:
    print(f"Model not found: {best_model_path}")

In [ ]:
# Test on random validation image
from ultralytics import YOLO
import random
import yaml
import os
import matplotlib.pyplot as plt

# Load trained model
best_model_path = "/content/drive/MyDrive/Uskudar Uni Master/YOLO Model klasor/runs/bosphorus_yolo11s_1088/weights/best.pt"

if not os.path.exists(best_model_path):
    print(f"Model not found: {best_model_path}")
else:
    trained_model = YOLO(best_model_path)
    
    dataset_path = "/content/drive/MyDrive/Uskudar Uni Master/YOLO Model klasor/Custom Dataset/bogaz_v_1.v3i.yolov12"
    yaml_path = os.path.join(dataset_path, "data.yaml")

    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    val_path = data.get('val')

    if val_path and os.path.exists(val_path):
        val_images = [f for f in os.listdir(val_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
        
        if val_images:
            random_image = random.choice(val_images)
            image_path = os.path.join(val_path, random_image)
            
            results = trained_model.predict(source=image_path, conf=0.25, imgsz=1088, save=True)
            
            for r in results:
                print(f"Detected {len(r.boxes)} objects")
                for box in r.boxes:
                    print(f"  - {r.names[int(box.cls[0])]}: {float(box.conf[0]):.2%}")
            
            annotated_img = results[0].plot()
            plt.figure(figsize=(16, 9))
            plt.imshow(annotated_img[:, :, ::-1])
            plt.axis('off')
            plt.show()

## Step 5: Export

In [ ]:
# Export model
from ultralytics import YOLO
import os
import shutil

best_model_path = "/content/drive/MyDrive/Uskudar Uni Master/YOLO Model klasor/runs/bosphorus_yolo11s_1088/weights/best.pt"
export_dir = "/content/drive/MyDrive/Uskudar Uni Master/YOLO Model klasor/exported_models"
os.makedirs(export_dir, exist_ok=True)

model = YOLO(best_model_path)

# Export to ONNX
onnx_path = model.export(format="onnx", imgsz=1088, simplify=True)

# Copy to export directory
shutil.copy(best_model_path, os.path.join(export_dir, "bosphorus_yolo11s_best.pt"))
if os.path.exists(onnx_path):
    shutil.copy(onnx_path, os.path.join(export_dir, "bosphorus_yolo11s_best.onnx"))

print(f"Exported to: {export_dir}")

In [ ]:
# Load in another notebook:
# model = YOLO("/content/drive/.../exported_models/bosphorus_yolo11s_best.pt")
# results = model.predict(source="image.jpg", conf=0.25)
# metrics = model.val(data="data.yaml")